In [1]:
#!pip install tensorflow-metal
import tensorflow as tf

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
def matmul_transpose(x, y):
    return tf.matmul(x, y, transpose_b=True)

def get_mats(i, j, k):
    return tf.random.normal((i, j)), tf.random.normal((k, j))

with tf.Graph().as_default() as graph:
    x, y = get_mats(2**12, 2**12, 2**12)
    output = matmul_transpose(x, y)
    with tf.compat.v1.Session(graph=graph) as sess:
        bm = tf.test.Benchmark()
        bm_result = bm.run_op_benchmark(sess, output)

2022-03-05 03:40:32.399282: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-05 03:40:32.399957: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-05 03:40:32.400351: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-05 03:40:32.411601: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_

Metal device set to: AMD Radeon Pro 555X

systemMemory: 16.00 GB
maxCacheSize: 2.00 GB



2022-03-05 03:41:10.600032: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-05 03:41:10.600054: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-05 03:41:12.160385: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-03-05 03:41:12.161297: W tensorflow/core/profiler/convert/xplane_to_step_stats.cc:74] GPU trace was not collected.
2022-03-05 03:41:12.161547: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


INFO:tensorflow:Benchmark [TensorFlowBenchmark.run_op_benchmark] iters: 10, wall_time: 3.60809, cpu_time: -1,throughput: 0, extras: {'allocator_maximum_num_bytes_cpu': 134217736, 'allocator_maximum_num_bytes_Simple allocator': 67108864}, metrics: None
entry {
  name: "TensorFlowBenchmark.run_op_benchmark"
  iters: 10
  wall_time: 3.608092427253723
  extras {
    key: "allocator_maximum_num_bytes_Simple allocator"
    value {
      double_value: 67108864.0
    }
  }
  extras {
    key: "allocator_maximum_num_bytes_cpu"
    value {
      double_value: 134217736.0
    }
  }
}



In [4]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])
predictions = model(x_train[:1]).numpy()
tf.nn.softmax(predictions).numpy()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

2022-03-05 03:41:24.061322: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-05 03:41:24.061353: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/5


2022-03-05 03:41:25.121113: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 19s 10ms/step - loss: 0.2875 - accuracy: 0.9162
Epoch 2/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.1373 - accuracy: 0.9591
Epoch 3/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.1017 - accuracy: 0.9694
Epoch 4/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0814 - accuracy: 0.9754
Epoch 5/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0705 - accuracy: 0.9782


2022-03-05 03:43:03.520385: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 - 3s - loss: 0.0813 - accuracy: 0.9742 - 3s/epoch - 8ms/step


[0.08133279532194138, 0.9741999506950378]